In [1]:
import pytmx
import mineworld.util as util
import mineworld.room as room
import mineworld.corridor as corridor
import os
import glob
import mcpi.minecraft as minecraft
import mcpi.entity as entity
import mcpi.block as block
import numpy as np
from mcpi.vec3 import Vec3

pygame 2.1.2 (SDL 2.0.18, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [21]:
# Load rooms that point all direction 
rooms = {}
for rf in glob.glob("rooms/*.pickle"):
    r = room.load_room(rf)
    rooms[r.name] = r
print(rooms)

{'sticker_room': <mineworld.room.Room object at 0x7facbf3dabf0>, 'factory': <mineworld.room.Room object at 0x7facbf594ac0>, 'castle_dungeon': <mineworld.room.Room object at 0x7facbf5c5db0>, 'dining': <mineworld.room.Room object at 0x7facbf5b1f60>, 'school': <mineworld.room.Room object at 0x7facbf5983d0>, 'war_room': <mineworld.room.Room object at 0x7facbf3b56c0>, 'miners_cave': <mineworld.room.Room object at 0x7facbf3bf610>, 'pokemon_room': <mineworld.room.Room object at 0x7facbf3a3580>, 'dice_room': <mineworld.room.Room object at 0x7facbf380fd0>, 'spider_room': <mineworld.room.Room object at 0x7facbd9d0c10>, 'library': <mineworld.room.Room object at 0x7facbf35e560>, 'lazerforce': <mineworld.room.Room object at 0x7facbf37f7f0>, 'castle': <mineworld.room.Room object at 0x7facbf33cb20>, 'swimming_pool': <mineworld.room.Room object at 0x7facbf346560>, 'bathroom': <mineworld.room.Room object at 0x7facbf32e4d0>, 'train_station': <mineworld.room.Room object at 0x7facbf314430>, 'bank': <minew

In [26]:
mc = minecraft.Minecraft.create(address='192.168.1.38')
mc.player.getPos()

Vec3(-1.5,233.0,5.5)

In [27]:
# Plonk all the rooms in a row
normalise = {
    'down': 270,
    'left': 0,
    'up': 90,
    'right': 180,
}

i = 0
for b in rooms['castle_dungeon'].blocks:
    #if b[0]==block.WEIGHTED_PRESSURE_PLATE_LIGHT[0]:
    #    rooms['castle_dungeon'].blocks[i] = block.AIR
    if b[0]==block.TNT:
        rooms['castle_dungeon'].blocks[i] = block.STAINED_GLASS_BLUE
    i += 1

x = 0
coords = {}
for r in rooms:
    rooms[r].draw(mc, *util.to_world(x,0,0), rot=normalise[rooms[r].door_style])
    corridor.draw_corridor(mc, *util.to_world(x,0,-1), rot=0, holes=['x0', 'x1', 'z0', 'z1'])
    coords[r] = util.to_world(x,0,0)
    x += 1

print(coords)
    

{'sticker_room': [0, 0, 0], 'factory': [9, 0, 0], 'castle_dungeon': [18, 0, 0], 'dining': [27, 0, 0], 'school': [36, 0, 0], 'war_room': [45, 0, 0], 'miners_cave': [54, 0, 0], 'pokemon_room': [63, 0, 0], 'dice_room': [72, 0, 0], 'spider_room': [81, 0, 0], 'library': [90, 0, 0], 'lazerforce': [99, 0, 0], 'castle': [108, 0, 0], 'swimming_pool': [117, 0, 0], 'bathroom': [126, 0, 0], 'train_station': [135, 0, 0], 'bank': [144, 0, 0], 'clothes_shop': [153, 0, 0]}


In [28]:
# Edit out the room that explodes
mc.player.setPos(5,1,5)

In [32]:
# Write new rooms that point north this time
x = 0
for r in rooms:
    print('Processing %s...' % r)
    coords = util.to_world(x,0,0)
    side = 9
    blocks = mc.getBlocksWithNBT(coords[0], coords[1], coords[2], coords[0]+side-1, coords[1]+side-1, coords[2]+side-1)
    room_obj = room.Room(
        name=r,
        side=9,
        door_style='left',
        source_location=Vec3(x, 0, 0),
        version=1,
        blocks=blocks,
    )
    room_obj.write(dir='rooms2')
    x += 1


Processing sticker_room...
Processing factory...
Processing castle_dungeon...
Processing dining...
Processing school...
Processing war_room...
Processing miners_cave...
Processing pokemon_room...
Processing dice_room...
Processing spider_room...
Processing library...
Processing lazerforce...
Processing castle...
Processing swimming_pool...
Processing bathroom...
Processing train_station...
Processing bank...
Processing clothes_shop...


In [33]:
# Reload new rooms
rooms2 = {}
for rf in glob.glob("rooms2/*.pickle"):
    r = room.load_room(rf)
    rooms2[r.name] = r
print(rooms2)

{'sticker_room': <mineworld.room.Room object at 0x7facbf3dbb50>, 'factory': <mineworld.room.Room object at 0x7facc1198520>, 'castle_dungeon': <mineworld.room.Room object at 0x7facc11a9810>, 'dining': <mineworld.room.Room object at 0x7facc11b6b00>, 'school': <mineworld.room.Room object at 0x7facc11c3df0>, 'war_room': <mineworld.room.Room object at 0x7facc11dd120>, 'miners_cave': <mineworld.room.Room object at 0x7facc11ea410>, 'pokemon_room': <mineworld.room.Room object at 0x7facc11fb700>, 'dice_room': <mineworld.room.Room object at 0x7facc0f48a30>, 'spider_room': <mineworld.room.Room object at 0x7facc0ff1d20>, 'library': <mineworld.room.Room object at 0x7facc0fe6fe0>, 'lazerforce': <mineworld.room.Room object at 0x7facc0f8c340>, 'castle': <mineworld.room.Room object at 0x7facc0f79630>, 'swimming_pool': <mineworld.room.Room object at 0x7facc0f6dcf0>, 'bathroom': <mineworld.room.Room object at 0x7facc0f83c10>, 'train_station': <mineworld.room.Room object at 0x7facc0f0cf40>, 'bank': <minew

In [34]:
# Generate new images
for r in rooms2:
    rooms2[r].get_image_at_y().save('rooms2_images/%s.png' % r)